<a href="https://colab.research.google.com/github/freezer2019/Rakuten-Paris-Product-Classification/blob/main/Rakuten_modele(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers
!pip install torch torchvision
!pip install sentencepiece

In [6]:
import pandas as pd
import numpy as np 
import nltk
from nltk import WordPunctTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords
import seaborn as sns
import os
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from transformers import CamembertModel, CamembertTokenizer, CamembertConfig
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import torch.nn as nn 
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms, utils
import random
import cv2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import torch.nn.functional as F

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import sklearn.ensemble as ske

In [8]:
import sklearn.discriminant_analysis as d_a

In [ ]:
#LinearDiscriminantAnalysis(solver='svd', shrinkage=None, priors=None, n_components=None, store_covariance=False, tol=0.0001, covariance_estimator=None)

In [9]:
import time
import copy

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [ ]:
from google.colab import drive,files
drive.mount("/content/drive")

In [ ]:
!unzip /content/drive/MyDrive/DS2021/Datasets/archive\(8\).zip 

In [11]:
x_train = pd.read_csv("/content/X_train_update.csv")
y_train = pd.read_csv("/content/Y_train_CVw08PX.csv")
x_test = pd.read_csv("/content/X_test_update.csv")

In [12]:
x_train.rename(columns = {"Unnamed: 0": "Id"}, inplace = True)
y_train.rename(columns = {"Unnamed: 0": "Id"}, inplace = True)
x_test.rename(columns = {"Unnamed: 0": "Id"}, inplace = True)
x_train["root_dir"]=x_train["Id"].apply(lambda x: "/content/images/images/image_train")
x_test["root_dir"]=x_train["Id"].apply(lambda x: "/content/images/images/image_test")


In [13]:
x_train.head()

,Id,designation,description,productid,imageid,root_dir
0,0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,/content/images/images/image_train
1,1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,/content/images/images/image_train
2,2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,/content/images/images/image_train
3,3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,/content/images/images/image_train
4,4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,/content/images/images/image_train


Helpers function, Classes

```
# This is formatted as code
```



In [14]:
class simple_Text_cleaner(BaseEstimator, TransformerMixin):
  # add another additional parameter, just for fun, while we are at it
    def __init__(self, stopwords,columns,tokenizer=WordPunctTokenizer()): 
        self.columns = columns
        self.stopwords = stopwords
        self.tokenizer=tokenizer
    def rm_stopwords(self,tokens):
        tokens=self.tokenizer.tokenize(tokens)
        return [ tk for tk in tokens if  tk not in self.stopwords ]
    def text_clean_up(self,s=""):
        import re
        user_pattern       = '@[^\s]+'
        s=re.sub(user_pattern, "", s)
        remove = '"#$%&()*+/:;<=>@[\\]^_`{|}~”“'
        pattern = r"[{}]".format(remove)
        s=re.sub(pattern,' ', s) 
        sequencePattern   = r"(.)\1\1+"
        seqReplacePattern = r"\1\1"
        s = re.sub(sequencePattern, seqReplacePattern, s)
        s = re.sub("<[^>]*>",' ', s)
        s = re.sub("[\r\n]+",' ', s)
        s = re.sub("http\S+",' ', s)
        s = re.sub("\$[^>]*\$",' ', s)
        s = re.sub("\d+",' ', s)
        s = re.sub("\s\s+",' ', s)
        s.strip()
        return s
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_ = X.copy()
        X_ ["image_path"]=X.apply(lambda x: generate_path(x["root_dir"], x["imageid"], x["productid"]),axis=1)
        for i in self.columns:
            X_[i] = X_[i].apply(lambda x: " ".join(self.rm_stopwords(self.text_clean_up(x))))
        return X_.drop(["root_dir", "imageid", "productid"], axis=1)

In [43]:
def encode_reviews(tokenizer, model, device, cpu, reviews, max_length,batch):
    token_ids = torch.tensor([])
    input_ids=torch.tensor([],dtype=torch.long)
    attention_mask=torch.tensor([],dtype=torch.long)
    h = 0
    for i, review in enumerate(reviews):
        encoded = encoded_text = tokenizer.encode_plus(
                        review,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_length,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
        input_ids=torch.cat((input_ids,encoded_text['input_ids']),dim=0)
        attention_mask=torch.cat((attention_mask,encoded_text['attention_mask']),dim=0)
        h+=1
        if h == batch:
            hidden = model(input_ids.to(device),attention_mask=attention_mask.to(device))[2][-2]
            token_ids=torch.cat((token_ids,torch.mean(hidden, dim=1).to(cpu)),dim=0)
            h=0
            input_ids=torch.tensor([],dtype=torch.long)
            attention_mask=torch.tensor([],dtype=torch.long)
    if len(input_ids) != 0:
        hidden = model(input_ids.to(device),attention_mask=attention_mask.to(device))[2][-2]
        token_ids=torch.cat((token_ids,torch.mean(hidden, dim=1).to(cpu)),dim=0)
        print(token_ids.size())
    token_ids=token_ids.numpy()
    print(token_ids.shape)
    return token_ids
def encode_text(tokenizer, model, device, cpu, text, max_length,batch):
    encoded = encoded_text = tokenizer.encode_plus(
                    text,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = max_length,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',     # Return pytorch tensors.
                )
    input_ids=encoded_text['input_ids']
    attention_mask=encoded_text['attention_mask']
    hidden = model(input_ids.to(device),attention_mask=attention_mask.to(device))[2]
    x=hidden[-2]
    #x=torch.cat((hidden[-4], hidden[-3], hidden[-2], hidden[-1]), dim = 2)
    token_ids=torch.mean(x, dim=1).to(cpu).squeeze(0)
    token_ids=token_ids.numpy()
    return token_ids

In [16]:
class CamembertPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, max_seq_length,column,bs=128):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.cpu=torch.device("cpu")
        print(self.device)
        self.tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
        self.config = CamembertConfig.from_pretrained("camembert-base", output_hidden_states=True)
        self.camembert =CamembertModel.from_pretrained("camembert-base", config=self.config).to(self.device)
        for p in self.camembert.parameters():
                p.requires_grad_(False)
        self.max_seq_length = max_seq_length
        self.column = column
        self.camembert.eval()
        self.batch=bs
    def fit(self, X=None):
        return self 
    
    def transform(self, X, y=None):
        # 1. Tokenize
        since = time.time()
        #X_encoded = encode_reviews(self.tokenizer,self.camembert,self.device,self.cpu, X[self.column].values, self.max_seq_length,self.batch)
        if y is None:
            seq_length=64
        else:
            seq_length=self.max_seq_length
        X_encoded = X[self.column].apply(lambda x: pd.Series(encode_text(self.tokenizer,self.camembert,self.device,self.cpu, x, seq_length,self.batch)))
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        X_encoded=np.array(X_encoded)
        print(len(X_encoded))
        print(len(X_encoded[0]))
        print(X_encoded.shape)
        return X_encoded  
    
    def fit_transform(self, X, y=None):        
        return self.transform(X, y)

In [ ]:

torch.cat((torch.tensor([]),torch.tensor([1,2])),dim=0)

In [17]:
class Second_to_last_SentenceEmbedding(nn.Module):
    def __init__(self,to_tune):
        #super(, self).__init__()
        super().__init__()
        self.config = CamembertConfig.from_pretrained("camembert-base", output_hidden_states=True)
        self.camembert = CamembertModel.from_pretrained("camembert-base", config=self.config)
        dense_layer1 = nn.Sequential(nn.Linear(768,100),nn.ReLU(),nn.Dropout(p=0.2))
        dense_layer2 = nn.Sequential(nn.Linear(100,54),nn.ReLU(),nn.Dropout(p=0.2))
        fc2=nn.Linear(54,27)
        self.dense=nn.Sequential(dense_layer1, dense_layer2, fc2)
        if not to_tune:
            for p in self.camembert.parameters():
                p.requires_grad_(False)
    def forward(self, input,attention_mask=None):
        hidden = self.camembert(input,attention_mask=attention_mask)[2]
        token_vecs = hidden[-2]
        x = torch.mean(token_vecs,dim=1)
        return self.dense(x)

In [18]:
class Columns_Selector(BaseEstimator, TransformerMixin):
    def __init__(self,column):
        self.column = column
    def fit(self, X = None):
        return self 
    def transform(self, X, y=None):
        return X[self.column]     
    def fit_transform(self, X, y=None):        
        return self.transform(X, y)

In [19]:
def generate_path(path,image_id,product_id):
    return os.path.join(path,f"image_{image_id}_product_{product_id}.jpg")
class Image_path(BaseEstimator, TransformerMixin):
    def __init__(self,column):
        self.column = column
    def fit(self, X = None):
        return self 
    def transform(self, X, y=None):
        X["image_path"]=X.apply(lambda x: generate_path(x["root_dir"], x["imageid"], x["productid"]),axis=1)
        X.drop(["root_dir", "imageid", "productid"], axis=1)
        return X.drop(["root_dir", "imageid", "productid"], axis=1)
    def fit_transform(self, X, y=None):        
        return self.transform(X, y)

In [20]:
class To_dense(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X = None):
        return self 
    def transform(self, X, y=None):
        X = X.toarray()
        print(X.shape)
        return X 
    def fit_transform(self, X, y=None):        
        return self.transform(X, y)

In [21]:
class MultimodalDataset (Dataset):
    def __init__(self,tokenizer, input_csv_file, root_dir,max_length=60 , transform=None,Y_csv_file=None):
        if type(input_csv_file) == str:
            self.input_file = pd.read_csv(input_csv_file)
        else:
            self.input_file = input_csv_file
        self.input_file.rename(columns={"Unnamed: 0": "Id"}, inplace=True)
        self.root_dir = root_dir
        self.transform =transform
        self.max_length=max_length
        self.input_file["image_path"]=(self.input_file["imageid"].apply(lambda x: str(int(x)))+"_product_"+self.input_file["productid"].apply(lambda x: str(int(x)))+".jpg").apply(lambda x: "image_"+x)
        if Y_csv_file is not None:
            if type(Y_csv_file) == str:
                self.output = pd.read_csv(Y_csv_file)
                
            else:
                self.output = Y_csv_file
            self.output.rename(columns={"Unnamed: 0": "Id"}, inplace=True)
            self.classes = list(set(self.output["prdtypecode"].values))
        else:
            self.ouptut = None
    def __len__(self):
        return len(self.input_file)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        try:
            img_name = os.path.join(self.root_dir,
                                    self.input_file["image_path"].iloc[idx])
        except:
            print(idx)

        image = io.imread(img_name)
        text = self.input_file["designation"].iloc[idx]
        encoded_text = tokenizer.encode_plus(
                        text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
        sample = {'image': image, 'input_ids':encoded_text['input_ids'].squeeze(0) ,'attention_mask':encoded_text['attention_mask'].squeeze(0)}
        sample['Id']=self.input_file["Id"].iloc[idx]
        if self.transform:
            sample['image'] = self.transform(sample['image'])
        if self.output is None:
            return sample
        sample["label"]=self.classes.index(self.output["prdtypecode"].loc[self.input_file["Id"].iloc[idx]])
        return sample

In [22]:
def encoder_multimodal(tokenizer, model, device, cpu, x, max_length,transform):
    encoded = encoded_text = tokenizer.encode_plus(
                    x["designation"],                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = max_length,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',     # Return pytorch tensors.
                )
    input_ids = encoded_text['input_ids']
    attention_mask = encoded_text['attention_mask']
    image = transform(io.imread(x["image_path"]))
    hidden = torch.cat(model(input = input_ids.to(device),
                             image = image.to(device),
                             attention_mask = attention_mask.to(device)),dim=1)
    
    token_ids = hidden.numpy()
    return token_ids

In [23]:
class Microscope:
    """
    Cutting out the edges around the center circle of the image
    Imitating a picture, taken through the microscope

    Args:
        p (float): probability of applying an augmentation
    """

    def __init__(self, p: float = 0.5):
        self.p = p

    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to apply transformation to.

        Returns:NumPyNumPy
            PIL Image: Image with transformation.
        """
        if random.random() < self.p:
            circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8), # image placeholder
                        (img.shape[0]//2, img.shape[1]//2), # center point of circle
                        random.randint(img.shape[0]//2 - 3, img.shape[0]//2 + 15), # radius
                        (0, 0, 0), # color
                        -1)

            mask = circle - 255
            img = np.multiply(img, mask)
        
        return img

    def __repr__(self):
        return f'{self.__class__.__name__}(p={self.p})'

In [24]:
class Feature_Extraction(nn.Module):
    # deuxieme dimension de l'output de chaque tenseurs a la sortie d'un layer de resnet
    resnet_caracteristics=[64,64,64,64,256,512,1024,2048,2048]
    pooling_target=[(4,8),(4,8),(4,8),(4,8),(2,4),(2,2),(1,2),(1,1),(1,1)]
    embedding_strategy={"Second_to_last_average":768,"Start_token_embedding":768,"last_four_embedding_average":3072}
    def __init__(self,resnet_layers=5,to_tune=False,sentence_embedding="Second_to_last_average"):
        #super(, self).__init__()
        super().__init__()
        self.config = CamembertConfig.from_pretrained("camembert-base", output_hidden_states=True)
        self.camembert =CamembertModel.from_pretrained("camembert-base", config=self.config)
        self.Resnet = models.resnet50(pretrained=True)
        self.Resnet = nn.Sequential(*list(self.Resnet.children())[:resnet_layers])
        self.pooling = nn.AdaptiveAvgPool2d(self.pooling_target[resnet_layers-1])
        self.flat = nn.Flatten()
        self.strategy = sentence_embedding
        if sentence_embedding not in self.embedding_strategy:
            self.strategy = "Second_to_last_average"    
        if self.strategy == "Start_token_embedding":
            to_tune = True
        if to_tune == False:
            for p in self.camembert.parameters():
                p.requires_grad_(False)
        for p in self.Resnet.parameters():
                p.requires_grad_(False)
        self.output_size = [2048, self.embedding_strategy[self.strategy]]
    def sentence_embedding(self,hiddens):
        # we get the output of the second to last hidden layers and average it over all token
        if self.strategy == "Second_to_last_average":
            return torch.mean(hiddens[-2],dim=1) 
        # we use the first token embedding  from the ouptut last hidden layers 
        # fine_tune should be true 
        elif self.strategy == "Start_token_embedding":
            return hiddens[-1].permute(1,0,2)[0]
        # we use the last four hidden layer average and concatenate them
        elif self.strategy == "last_four_embedding_average":
            x=torch.cat((hiddens[-4], hiddens[-3], hiddens[-2], hiddens[-1]), dim = 2)
            return torch.mean(x, dim=1)
    def forward(self, input, image, attention_mask=None):
        hiddens = self.camembert(input,attention_mask = attention_mask)[2]
        embeddings = self.sentence_embedding(hiddens)
        x = self.Resnet(image)
        h = self.pooling(x)
        return embeddings, self.flat(h)

In [25]:
class Camembert_resnet_Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, max_seq_length,resnet_layers=5,to_tune=False,sentence_embedding="Second_to_last_average"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.cpu=torch.device("cpu")
        print(self.device)
        self.tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
        self.feature_extractor = Feature_Extraction(resnet_layers = resnet_layers,
                                                    to_tune = to_tune,
                                                    sentence_embedding = sentence_embedding).to(self.device)
        self.max_seq_length = max_seq_length
        self.feature_extractor.eval()
        self.data_transform=transforms.Compose([transforms.ToPILImage(),
                                    transforms.Resize(256),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])
    def fit(self, X=None):
        return self 
    
    def transform(self, X, y=None):
        # 1. Tokenize
        since = time.time()
        #X_encoded = encode_reviews(self.tokenizer,self.camembert,self.device,self.cpu, X[self.column].values, self.max_seq_length,self.batch)
        if y is None:
            seq_length=64
        else:
            seq_length=self.max_seq_length
        X_encoded = X.apply(lambda x: pd.Series(encoder_multimodal(self.tokenizer,self.feature_extractor,self.device,self.cpu, x, seq_length,self.data_transform)))
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        X_encoded=np.array(X_encoded)
        print(len(X_encoded))
        print(len(X_encoded[0]))
        print(X_encoded.shape)
        return X_encoded  
    
    def fit_transform(self, X, y=None):        
        return self.transform(X, y)

In [26]:
class Multimodal_Dense_model(nn.Module):
    def __init__(self,dropout=0.2,resnet_layers=-1,to_tune=False,sentence_embedding="Second_to_last_average"):
        #super(, self).__init__()
        super().__init__()
        self.feature_extractor = Feature_Extraction(resnet_layers=resnet_layers,to_tune=to_tune,sentence_embedding=sentence_embedding)
        self.input_size  = self.feature_extractor.output_size
        self.dense_layer = nn.Sequential(OrderedDict([
          ('dense1', nn.Linear(in_features=sum(self.input_size),out_features=1024)),
          ('relu1', nn.ReLU()),
          ('dropout1', nn.Dropout(p=0.1)),
          ('dense2', nn.Linear(in_features=1024,out_features=256)),
          ('relu2', nn.ReLU()),
          ('dropout2', nn.Dropout(p=0.1)),
          ('dense3', nn.Linear(in_features=256,out_features=64)),
          ('relu3', nn.ReLU()),
          ('dropout3', nn.Dropout(p=0.1)),
          ('output', nn.Linear(64,27))
          ]))
    def forward(self, input,image,attention_mask=None):
        self.feature_extractor.eval()
        text,image = self.feature_extractor(input=input,image=image,attention_mask=attention_mask)
        return self.dense_layer(torch.cat((text,image),dim=1))

In [27]:
data_transforms = {
    'train': transforms.Compose([
        Microscope(p=0.5),
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(size=384, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [28]:
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
training_set = MultimodalDataset(tokenizer=tokenizer, input_csv_file="/content/X_train_update.csv", root_dir="/content/images/images/image_train",max_length=60 , transform=data_transforms["train"],Y_csv_file="/content/Y_train_CVw08PX.csv")
def make_weights_for_balanced_classes(images, nclasses):                                                                            
    count = list(images.output["prdtypecode"].value_counts().loc[images.classes])                                    
    N = float(sum(count)) 
    weight_per_class=[0]*nclasses                                                  
    for i in range(nclasses):                                                   
        weight_per_class[i] = N/float(count[i])                                 
    weight = [0] * len(images)                                              
    for idx in range(int(N)):    
        h=images.classes.index(images.output["prdtypecode"].iloc[idx])                                      
        weight[idx] = weight_per_class[h]                                  
    return weight
weights = make_weights_for_balanced_classes(training_set, len(training_set.classes))                                                                
weights = torch.DoubleTensor(weights)                                       
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))                     
                                                                                                                                                                        
train_dl = torch.utils.data.DataLoader(training_set, batch_size=16,                              
                                                             sampler = sampler, num_workers=4)

Baseline model
Tf-idfVectorizer + Logistics Regression (Texte)

In [29]:
X_train,X_Val,Y_train,Y_Val=train_test_split(x_train, y_train, test_size = 0.1, shuffle = True , random_state = 102)

In [30]:
target = Y_train["prdtypecode"]

In [ ]:
?LogisticRegression

In [31]:
stop_words = stopwords.words('french')
stop_words.extend(stopwords.words('english'))

In [ ]:
Base_model = Pipeline(steps = [('cleaner', simple_Text_cleaner(stopwords =stop_words, columns = ["designation"])),
                               ('column_selector', Columns_Selector(column = "designation")),
                               ('Vectorization', TfidfVectorizer(stop_words = stop_words, max_features = 5000)),
                               ('to_dense', To_dense()),
                               ('scaler', StandardScaler()),
                               ('model', LogisticRegression(multi_class='multinomial',class_weight= "balanced"))
                           ])

In [ ]:
Base_model.fit(X_train,target)

(76424, 5000)


Pipeline(memory=None,
         steps=[('cleaner',
                 simple_Text_cleaner(columns=['designation'],
                                     stopwords=['au', 'aux', 'avec', 'ce',
                                                'ces', 'dans', 'de', 'des',
                                                'du', 'elle', 'en', 'et', 'eux',
                                                'il', 'ils', 'je', 'la', 'le',
                                                'les', 'leur', 'lui', 'ma',
                                                'mais', 'me', 'même', 'mes',
                                                'moi', 'mon', 'ne', 'nos', ...],
                                     tokenizer=WordPunctTokenizer(pattern='\\w+|[^\\w\\s]+', gaps=False, discard_empt...
                ('to_dense', To_dense()),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 LogisticRegression(C=1.0, class_we

In [ ]:
pred = Base_model.predict(X_Val)

(8492, 5000)


In [ ]:
print(classification_report(Y_Val["prdtypecode"],pred))

              precision    recall  f1-score   support

          10       0.26      0.23      0.24       301
          40       0.59      0.55      0.57       259
          50       0.72      0.71      0.72       186
          60       0.83      0.80      0.82        97
        1140       0.69      0.63      0.66       254
        1160       0.86      0.86      0.86       362
        1180       0.43      0.49      0.46        67
        1280       0.58      0.54      0.56       491
        1281       0.36      0.44      0.40       193
        1300       0.87      0.85      0.86       549
        1301       0.90      0.88      0.89        90
        1302       0.73      0.73      0.73       258
        1320       0.67      0.65      0.66       325
        1560       0.74      0.75      0.74       488
        1920       0.85      0.85      0.85       419
        1940       0.66      0.67      0.67        89
        2060       0.73      0.73      0.73       485
        2220       0.67    

[link text](https:// [link text](https://))Bert extraction De feature 

In [ ]:
Simple_text_model = Pipeline(steps = [('cleaner', simple_Text_cleaner(stopwords =stop_words, columns = ["designation"])),
                               ('Vectorization', CamembertPreprocessor(max_seq_length =40 , column = "designation")),
                               ('scaler', StandardScaler()),
                               ('model', LogisticRegression(multi_class='multinomial',class_weight= "balanced"))
                           ])

cuda


In [ ]:
Simple_text_model.fit(X_train,target)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training complete in 17m 19s
76424
3072
(76424, 3072)


Pipeline(memory=None,
         steps=[('cleaner',
                 simple_Text_cleaner(columns=['designation'],
                                     stopwords=['au', 'aux', 'avec', 'ce',
                                                'ces', 'dans', 'de', 'des',
                                                'du', 'elle', 'en', 'et', 'eux',
                                                'il', 'ils', 'je', 'la', 'le',
                                                'les', 'leur', 'lui', 'ma',
                                                'mais', 'me', 'même', 'mes',
                                                'moi', 'mon', 'ne', 'nos', ...],
                                     tokenizer=WordPunctTokenizer(pattern='\\w+|[^\\w\\s]+', gaps=False, discard_empt...
                                       max_seq_length=40)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 LogisticRegressi

In [ ]:
pred = Simple_text_model.predict(X_Val)

Training complete in 1m 56s
8492
3072
(8492, 3072)


In [ ]:
print(classification_report(Y_Val["prdtypecode"],pred))

              precision    recall  f1-score   support

          10       0.46      0.48      0.47       301
          40       0.52      0.60      0.56       259
          50       0.70      0.76      0.73       186
          60       0.86      0.72      0.79        97
        1140       0.57      0.62      0.60       254
        1160       0.87      0.86      0.86       362
        1180       0.31      0.57      0.40        67
        1280       0.66      0.41      0.51       491
        1281       0.38      0.50      0.43       193
        1300       0.90      0.86      0.88       549
        1301       0.84      0.81      0.82        90
        1302       0.60      0.74      0.66       258
        1320       0.61      0.64      0.63       325
        1560       0.75      0.71      0.73       488
        1920       0.88      0.85      0.86       419
        1940       0.75      0.75      0.75        89
        2060       0.77      0.69      0.73       485
        2220       0.52    

In [ ]:
print(classification_report(Y_train["prdtypecode"],Simple_text_model.predict(X_train)))

Training complete in 17m 16s
76424
3072
(76424, 3072)
              precision    recall  f1-score   support

          10       0.66      0.72      0.69      2815
          40       0.70      0.82      0.75      2249
          50       0.81      0.89      0.85      1495
          60       0.97      0.88      0.92       735
        1140       0.73      0.81      0.77      2417
        1160       0.91      0.90      0.90      3591
        1180       0.49      0.89      0.63       697
        1280       0.77      0.48      0.59      4379
        1281       0.55      0.67      0.60      1877
        1300       0.92      0.90      0.91      4496
        1301       0.96      0.97      0.97       717
        1302       0.67      0.88      0.76      2233
        1320       0.71      0.80      0.76      2916
        1560       0.83      0.79      0.81      4585
        1920       0.90      0.88      0.89      3884
        1940       0.87      0.98      0.92       714
        2060       0.84    

In [ ]:
print(classification_report(Y_train["prdtypecode"],Base_model.predict(X_train)))

(76424, 5000)
              precision    recall  f1-score   support

          10       0.74      0.64      0.69      2815
          40       0.91      0.88      0.90      2249
          50       0.98      0.98      0.98      1495
          60       0.98      0.99      0.99       735
        1140       0.93      0.94      0.94      2417
        1160       0.98      0.96      0.97      3591
        1180       0.80      0.95      0.87       697
        1280       0.95      0.87      0.91      4379
        1281       0.81      0.89      0.85      1877
        1300       0.98      0.97      0.98      4496
        1301       0.99      1.00      0.99       717
        1302       0.96      0.97      0.97      2233
        1320       0.96      0.96      0.96      2916
        1560       0.98      0.96      0.97      4585
        1920       0.97      0.99      0.98      3884
        1940       0.89      0.97      0.93       714
        2060       0.98      0.96      0.97      4508
        2220 

In [ ]:
Simple_text_sgdmodel = Pipeline(steps = [('cleaner', simple_Text_cleaner(stopwords =stop_words, columns = ["designation"])),
                               ('Vectorization', CamembertPreprocessor(max_seq_length = 32, column = "designation")),
                               ('scaler', StandardScaler()),
                               ('model', SGDClassifier(class_weight= "balanced"))
                           ])

cuda


In [ ]:
Simple_text_sgdmodel.fit(X_train,target)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training complete in 17m 11s
76424
3072
(76424, 3072)


Pipeline(memory=None,
         steps=[('cleaner',
                 simple_Text_cleaner(columns=['designation'],
                                     stopwords=['au', 'aux', 'avec', 'ce',
                                                'ces', 'dans', 'de', 'des',
                                                'du', 'elle', 'en', 'et', 'eux',
                                                'il', 'ils', 'je', 'la', 'le',
                                                'les', 'leur', 'lui', 'ma',
                                                'mais', 'me', 'même', 'mes',
                                                'moi', 'mon', 'ne', 'nos', ...],
                                     tokenizer=WordPunctTokenizer(pattern='\\w+|[^\\w\\s]+', gaps=False, discard_empt...
                 SGDClassifier(alpha=0.0001, average=False,
                               class_weight='balanced', early_stopping=False,
                               epsilon=0.1, eta0=0.0, fit_intercept=True,
          

In [ ]:
print("============Bert============")
print(classification_report(Y_Val["prdtypecode"],Simple_text_sgdmodel.predict(X_Val),))

============Bert============
Training complete in 1m 52s
8492
3072
(8492, 3072)
              precision    recall  f1-score   support

          10       0.53      0.39      0.45       301
          40       0.47      0.47      0.47       259
          50       0.48      0.74      0.58       186
          60       0.91      0.63      0.74        97
        1140       0.58      0.56      0.57       254
        1160       0.83      0.76      0.79       362
        1180       0.17      0.63      0.27        67
        1280       0.53      0.40      0.45       491
        1281       0.28      0.41      0.34       193
        1300       0.94      0.61      0.74       549
        1301       0.77      0.80      0.79        90
        1302       0.53      0.65      0.58       258
        1320       0.42      0.74      0.54       325
        1560       0.87      0.53      0.66       488
        1920       0.83      0.81      0.82       419
        1940       0.62      0.79      0.69        89
 

In [ ]:
from sklearn.naive_bayes import MultinomialNB



> Indented block
Random Forest Model 
using Bert 

In [44]:
Simple_text_xgbmodel = Pipeline(steps = [('cleaner', simple_Text_cleaner(stopwords =stop_words, columns = ["designation"])),
                               ('Vectorization', CamembertPreprocessor(max_seq_length = 40, column = "designation")),
                               ('scaler', StandardScaler()),
                               ('lda',d_a.LinearDiscriminantAnalysis(n_components=100)),
                               ('model', RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=10, verbose=1,class_weight="balanced"))
                           ])

cuda


In [ ]:
Simple_text_xgbmodel.fit(X_train,target)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training complete in 15m 21s
76424
768
(76424, 768)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


In [ ]:
print("============Bert============")
print(classification_report(Simple_text_xgbmodel.predict(X_Val),Y_Val["prdtypecode"]))

Perceptron Model

General Train Function-

In [ ]:
soft_max = nn.LogSoftmax(dim=1)
import numpy as np
from sklearn.metrics import f1_score
def accuracy(out, yb):
    preds = torch.argmax(soft_max(out), dim=1) # classe prédite: tensor de taille bs. 
    score=f1_score(yb.to(cpu).numpy(),preds.to(cpu).numpy(), average="weighted")
    return score
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    all_loss={"train":[],"val":[]}
    all_acc={"train":[],"val":[]}
    all_acc_all={"train":[],"val":[]}
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            all_labels=torch.tensor([],dtype=torch.float32)
            all_ouputs=torch.tensor([],dtype=torch.float32).to(device)
            # Iterattention_maskate over data.
            for inputs in dataloaders[phase]:
                input_ids = inputs["input_ids"].to(device)
                attention_mask = inputs["attention_mask"].to(device)
                all_labels=torch.cat((all_labels,inputs["label"]),dim=0)
                labels = inputs["label"].to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(input_ids, attention_mask = attention_mask)
                    loss = criterion(outputs, labels , weight  = torch.tensor(weights,dtype = torch.float32).to(device))
                    all_ouputs=torch.cat((all_ouputs,outputs),dim=0)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * input_ids.size(0)
                running_corrects += accuracy(outputs ,labels)* input_ids.size(0)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataset[phase])
            epoch_acc = running_corrects / len(dataset[phase])
            epoch_acc_all = accuracy(all_ouputs ,all_labels)
            all_loss[phase].append(epoch_loss)
            all_acc[phase].append(epoch_acc)
            all_acc_all[phase].append(epoch_acc_all)
            print('{} Loss: {:.4f} Acc: {:.4f} , Acc_all: {:.4f}'.format(
                phase, epoch_loss, epoch_acc,epoch_acc_all))

            # deep copy the model
            if phase == 'val' and epoch_acc_all > best_acc:
                best_acc = epoch_acc_all
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, all_loss, all_acc,all_acc_all

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu=torch.device("cpu")

Torch Dataset et DataLoaders

In [ ]:
dataset={}
dataloaders={}
dataset["full"] = MultimodalDataset(tokenizer=tokenizer, input_csv_file="/content/X_train_update.csv", root_dir="/content/images/images/image_train",max_length=64 , transform=data_transforms["train"],Y_csv_file="/content/Y_train_CVw08PX.csv")                     
dataloaders["full"] = torch.utils.data.DataLoader(dataset["full"], batch_size=128,shuffle=True, num_workers=6)#balanced dataset

dataset["train"]=MultimodalDataset(tokenizer=tokenizer, input_csv_file=X_train, root_dir="/content/images/images/image_train",max_length=108 , transform=data_transforms["train"],Y_csv_file=Y_train)         
dataloaders["train"] = torch.utils.data.DataLoader(dataset["train"], batch_size=128,shuffle=True, num_workers=6)#balanced dataset

dataset["val"]=MultimodalDataset(tokenizer=tokenizer, input_csv_file=X_Val, root_dir="/content/images/images/image_train",max_length=108 , transform=data_transforms["train"],Y_csv_file=Y_Val)                                                           
dataloaders["val"] = torch.utils.data.DataLoader(dataset["val"], batch_size=128,shuffle=True, num_workers=6)#balanced dataset

**Text** Perceptron

```
# This is formatted as code
```
On va utiliser la classe Second_to_last_embeddings


In [ ]:
Simple_text_model_untuned=Second_to_last_SentenceEmbedding(to_tune=False).to(device)

In [ ]:
optimizer_untuned = optim.SGD(Simple_text_model_untuned.dense.parameters(), lr=0.005, momentum=0.9)

exp_lr_scheduler_untuned = lr_scheduler.StepLR(optimizer_untuned, step_size=7, gamma=0.1)

In [ ]:
loss_function= F.cross_entropy

training the model without tuning the Bert

In [ ]:
weights=compute_class_weight("balanced", classes= np.unique(y_train["prdtypecode"]), y=y_train["prdtypecode"])
h=np.unique(target)
weights_dict={}
for i in range(len(h)):
    weights_dict[h[i]] = weights[i]
    

In [ ]:
Simple_text_model_untuned,untuned_loss,untuned_acc,untuned_acc_all = train_model(Simple_text_model_untuned, 
                                                   loss_function, optimizer_untuned, exp_lr_scheduler_untuned, num_epochs=24)

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(10,10))
untuned_loss,untuned_acc
axs[0].plot(list(range(len(untuned_loss["train"]))),untuned_loss["train"],label="Train")
axs[0].plot(list(range(len(untuned_loss["val"]))),untuned_loss["val"],label="Valid")
axs[0].set_title("Loss")
axs[1].plot(list(range(len(untuned_acc["train"]))),untuned_acc["train"],label="Train")
axs[1].plot(list(range(len(untuned_acc["val"]))),untuned_acc["val"],label="Valid")
axs[1].set_title("F1_Score")
plt.legend()

Tuning Bert

Tuning Big Model 

In [ ]:
from collections import OrderedDict
model_1 = Multimodal_Dense_model(resnet_layers = 6, to_tune = True ,sentence_embedding = "last_four_embedding_average").to(device)

In [ ]:
optimizer_1 = optim.SGD(model_1.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler_1 = lr_scheduler.StepLR(optimizer_1, step_size=7, gamma=0.1)


In [ ]:
def train_model_multi(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    all_loss={"train":[],"val":[]}
    all_acc={"train":[],"val":[]}
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterattention_maskate over data.
            for inputs in dataloaders[phase]:
                image = inputs["image"].to(device)
                input_ids = inputs["input_ids"].to(device)
                attention_mask = inputs["attention_mask"].to(device)
                labels = inputs["label"].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(input_ids,image = image, attention_mask = attention_mask)
                    loss = criterion(outputs, labels, weight=torch.tensor(weights,dtype = torch.float32).to(device))

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * input_ids.size(0)
                running_corrects += accuracy(outputs ,labels)*input_ids.size(0)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataset[phase])
            epoch_acc = running_corrects.double() /len(dataset[phase])
            all_loss[phase].append(epoch_loss)
            all_acc[phase].append(epoch_acc)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, all_loss, all_acc

In [ ]:
model_1,loss_1,acc_1 = train_model_multi(model_1, loss_function, optimizer_1, exp_lr_scheduler_1, num_epochs=25)